In [179]:
## Import methods from
from hessenberg import *
from scipy.linalg import hessenberg as hessen

## Hessenberg Form Matrix

In [180]:
## Calculating different matrix for computing hessenberg form
A = np.array([[1,2,3,4], [5,6,7,8], [9, 8, 7, 6], [5,4, 3, 2]], dtype = float)
H = hessenberg_form(A)

In [183]:
## comparing the hessen output with implemented algorithm
inbuiltH = hessen(A)
print(np.linalg.norm(H - inbuiltH, "fro"))
print(np.linalg.norm(H - inbuiltH, 2))

2.1532218528415368e-14
2.0512945252629596e-14


## Converging the hessenberg form matrix to diagonal


* Research algorithms for phase 2